In [14]:
import robustbench
from tqdm import tqdm
import os
import torch
from autoattack import AutoAttack
from robustbench.data import _load_dataset
from torchvision import transforms, datasets
import robustbench
from robustbench.utils import load_model
import multiprocessing as mp
import pandas as pd
import sys
sys.path.append("./retrain_robustbench")
import data
import models
from glob import glob
from collections import defaultdict
import multiprocessing as mp
import numpy as np
import onnx
import struct
from joblib import Parallel, delayed
from glob import glob
import torchvision

In [21]:
gpus = [0, 1, 2, 3]
workers_per_gpu = 1

def gpu_worker(gpu, q):
    
    DEVICE = f"cuda:{gpu}"
    
    testset = torchvision.datasets.CIFAR10(root='/data/datasets/cifar10', train=False, download=True, transform=transforms.ToTensor())
    x_test, y_test = _load_dataset(testset)
    x_test, y_test = x_test.to(DEVICE), y_test.to(DEVICE)
    
    while not q.empty():
        file = q.get()
        model_name = os.path.abspath(os.path.join(file, '../../..'))
        model_name = model_name[model_name.rfind("/")+1:]
        model = models.get_model(model_name, dataset="cifar10")
        state = torch.load(file, map_location=DEVICE)
        model.load_state_dict(dict((key[len("model."):], value) for (key, value) in state["state_dict"].items()))        
        model.to(DEVICE)
        model.eval()
        adversary = AutoAttack(model, norm='Linf', eps=8/255, log_path=f"robust_benchmark/{model_name}_cifar10.log", device=DEVICE)
        adversary.run_standard_evaluation(x_test, y_test)
        
if __name__ == '__main__':
    que = mp.Queue()
    
    for file in glob("/ssd1/checkpoints_robustbench_models_normal_training/cifar10/*/*/checkpoints/*.ckpt"):
        que.put(file)
    
    processes = []
    for gpu in gpus * workers_per_gpu:
        p = mp.Process(target=gpu_worker, args=(gpu, que))
        p.start()
        processes.append(p)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
setting parameters for standard version
setting parameters for standard versionusing standard version including apgd-ce, apgd-t, fab-t, square

using standard version including apgd-ce, apgd-t, fab-t, square
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 94.51%
initial accuracy: 95.10%
initial accuracy: 95.64%initial accuracy: 95.35%

apgd-ce - 1/38 - 250 out of 250 successfully perturbed
apgd-ce - 2/38 - 250 out of 250 successfully perturbed
apgd-ce - 3/38 - 250 out of 250 successfully perturbed
apgd-ce - 4/38 - 250 out of 250 successfully perturbed
apgd-ce - 5/38 - 250 out of 250 successfully perturbed
apgd-ce - 6/38 - 250 out of 250 successfully perturbed
ap